In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torch

In [63]:
import torch.nn as nn
from torch import optim
from torch.nn import MSELoss
import torch.nn.functional as F

from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST

from sklearn.metrics import accuracy_score


ds = FashionMNIST(root="./", download=True, train=True, transform=transforms.ToTensor())
loader = DataLoader(ds, batch_size=32)

net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 100),
    nn.Sigmoid(),
    nn.Linear(100, 10),
    nn.Sigmoid()
).cuda()

opt = optim.SGD(net.parameters(), lr=1e-2)

criterion = MSELoss()  # From now on error = criterion(pred, y)

for epoch in range(30):
    errors, accuracy = [], []
    
    for x, y in loader:
        pred = net(x.cuda())
        
        y = F.one_hot(y, 10).to(torch.float32).cuda()
        
        err = criterion(pred, y)
        err.backward()
        
        pred_conv = torch.argmax(pred, dim=1).detach().cpu().numpy()
        y_conv = torch.argmax(y, dim=1).detach().cpu().numpy()
        
        accuracy.append(accuracy_score(pred_conv, y_conv))
        
        errors.append(err.item())
        
        opt.step()
        opt.zero_grad()
        
        
    print(f'Epoch: {epoch}, error: {sum(errors) / len(errors)}, accuracy: {sum(accuracy) / len(accuracy)}')
